# Visualización de la Distribución de Estudiantes por Sexo, Rama Educacional y Región

## 1. Importación de Librerías

En esta primera sección, importamos las librerías necesarias para manipular los datos y realizar las visualizaciones. Se utiliza **pandas** para la manipulación de datos y **plotly.express** para la creación de gráficos interactivos.

In [111]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


## 2. Carga de Datos

A continuación, cargamos los datos procesados del archivo CSV que contiene la información de los estudiantes inscritos en el proceso de admisión PAES 2025. Estos datos ya han sido previamente limpiados y estructurados en un archivo CSV.

In [112]:
# Función para cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

## 3. Mapeo de Variables

Para mejorar la legibilidad de los datos, se realiza el mapeo de valores numéricos a categorías. En esta sección, asignamos nombres descriptivos a las ramas educacionales y las regiones, así como a los valores correspondientes al sexo (1 = Hombre, 2 = Mujer).

In [113]:
data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})

### Mapeo de Ramas Educacionales:


In [114]:
# Diccionario para mapear ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

### Mapeo de Regiones:


In [115]:
# Diccionario para mapear regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

## 4. Cálculo de la Distribución

Calculamos la distribución de los estudiantes agrupados por sexo, rama educacional y región. También se calcula el porcentaje de cada grupo con respecto al total de estudiantes de cada sexo.

In [116]:
# Función para calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

# Calculamos la distribución
distribution = calcular_distribucion(data)

## 5. Visualización con Treemap

Utilizamos un gráfico treemap para visualizar la distribución de los estudiantes, clasificándolos por sexo, rama educacional y región. Cada cuadro en el gráfico representa un subconjunto de estudiantes, y el tamaño del cuadro está relacionado con el porcentaje de ese grupo dentro de su categoría.

Los colores se asignan según el sexo, con azul para los hombres y rosa para las mujeres, facilitando la identificación visual.

In [117]:
# Función para crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(
        distribution, 
        path=['general_label', 'sexo', 'label_with_percentage'], 
        values='percentage', 
        color='percentage',  
        color_continuous_scale='Viridis',  
        labels={'percentage': 'Distribución de estudiantes en %'},
        hover_data={'count': True}
    )
    
    # Personalización del diseño
    fig.update_layout(
        height=1000,  
        margin=dict(t=200, b=150),
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de<br>estudiantes en %</b>",  
                font=dict(size=18, family="Econ Sans Cnd")
            ),
            orientation='h',  
            x=0.79,  
            y=1.02,  
            thickness=17,
            len=0.42,
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],
            tickfont=dict(size=16),
            ticks="outside",
            tickwidth=1,
            ticklen=10
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,
                text="<b>Análisis de Admisiones PAES 2025</b>",
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Anotaciones inferiores
            dict(
                x=0, y=-0.05,
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.05,
                text="<b>Nota: Datos correspondientes al proceso de admisión 2025</b>",
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,
                line=dict(color="red", width=1),
                xref="paper", yref="paper"
            ),
            # Rectángulo decorativo
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustes adicionales al treemap
    fig.update_traces(
        marker=dict(line=dict(width=2)),
        textinfo='label+text+value',
        textfont_size=18,  
        texttemplate='%{label}<br>%{value}%',
        tiling=dict(pad=1, squarifyratio=1.0)
    )
    
    fig.show()

# Generamos el treemap
crear_treemap(distribution)

# Visualización de la Distribución de Estudiantes por Sexo, Rama Educacional y Región

## 1. Importación de Librerías

En esta primera sección, importamos las librerías necesarias para manipular los datos y realizar las visualizaciones. Se utiliza **pandas** para la manipulación de datos y **plotly.express** para la creación de gráficos interactivos.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go



## 2. Carga de Datos

A continuación, cargamos los datos procesados del archivo CSV que contiene la información de los estudiantes inscritos en el proceso de admisión PAES 2025. Estos datos ya han sido previamente limpiados y estructurados en un archivo CSV.

In [ ]:
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)


## 3. Mapeo de Variables

Para mejorar la legibilidad de los datos, se realiza el mapeo de valores numéricos a categorías. En esta sección, asignamos nombres descriptivos a las ramas educacionales y las regiones, así como a los valores correspondientes al sexo (1 = Hombre, 2 = Mujer).

In [ ]:
data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})


### Mapeo de Ramas Educacionales:
Convertimos los códigos de ramas educacionales en etiquetas descriptivas. Estas etiquetas permiten identificar de manera clara la especialización de los estudiantes.

In [ ]:
def mapear_columnas(data: pd.DataFrame) -> pd.DataFrame:
    # Mapeo de sexo
    data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})
    
    # Mapeo de ramas educacionales
    rama_educacional_mapping = {
        'H1': 'Humanista científico Diurno',
        'H2': 'Humanista científico Nocturno',
        'H3': 'Humanista científico - validación de estudios',
        'T1': 'Técnico profesional comercial',
        'T2': 'Técnico profesional Industrial'
    }
    data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)
    
    # Mapeo de regiones
    region_mapping = {
        15: 'Región de Arica y Parinacota',
        1: 'Región de Tarapacá',
        2: 'Región de Antofagasta',
        13: 'Región Metropolitana de Santiago',
        8: 'Región del Biobío'
    }
    data['codigo_region'] = data['codigo_region'].map(region_mapping)
    
    return data

data = mapear_columnas(data)


### Mapeo de Regiones:


In [ ]:
def mapear_columnas(data: pd.DataFrame) -> pd.DataFrame:
    # Mapeo de sexo
    data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})
    
    # Mapeo de ramas educacionales
    rama_educacional_mapping = {
        'H1': 'Humanista científico Diurno',
        'H2': 'Humanista científico Nocturno',
        'H3': 'Humanista científico - validación de estudios',
        'T1': 'Técnico profesional comercial',
        'T2': 'Técnico profesional Industrial'
    }
    data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)
    
    # Mapeo de regiones
    region_mapping = {
        15: 'Región de Arica y Parinacota',
        1: 'Región de Tarapacá',
        2: 'Región de Antofagasta',
        13: 'Región Metropolitana de Santiago',
        8: 'Región del Biobío'
    }
    data['codigo_region'] = data['codigo_region'].map(region_mapping)
    
    return data

data = mapear_columnas(data)


## 4. Cálculo de la Distribución

Calculamos la distribución de los estudiantes agrupados por sexo, rama educacional y región. También se calcula el porcentaje de cada grupo con respecto al total de estudiantes de cada sexo.

In [ ]:
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)


## 5. Visualización con Treemap

Utilizamos un gráfico treemap para visualizar la distribución de los estudiantes, clasificándolos por sexo, rama educacional y región. Cada cuadro en el gráfico representa un subconjunto de estudiantes, y el tamaño del cuadro está relacionado con el porcentaje de ese grupo dentro de su categoría.

Los colores se asignan según el sexo, con azul para los hombres y rosa para las mujeres, facilitando la identificación visual.

In [ ]:
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(
        distribution, 
        path=['general_label', 'sexo', 'label_with_percentage'], 
        values='percentage', 
        color='percentage', 
        color_continuous_scale='Viridis', 
        labels={'percentage': 'Distribución de estudiantes en %'}
    )
    
    # Estilización del gráfico
    fig.update_layout(
        height=1000,
        margin=dict(t=200, b=150),
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de<br>estudiantes en %</b>",
                font=dict(size=18, family="Econ Sans Cnd")
            ),
            orientation='h',
            x=0.79,
            y=1.02,
            thickness=17,
            len=0.42,
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],
            tickfont=dict(size=16),
            ticks="outside",
            tickwidth=1,
            ticklen=10
        ),
        annotations=[
            dict(
                x=0, y=1.15,
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper"
            ),
            dict(
                x=0, y=-0.05,
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper"
            )
        ]
    )
    
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    color_map = {
        'Hombre': '#1f77b4',  # Azul para hombres
        'Mujer': 'pink'       # Rosa para mujeres
    }
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='sexo', 
                     color_discrete_map=color_map,
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>All work, no play</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Children in child labour*</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores
            dict(
                x=0.01, y=-0.15,
                text='Selected regions, % of child population',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.15,
                text="Number of children, m",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    color_map = {
        'Hombre': '#1f77b4',  # Azul para hombres
        'Mujer': 'pink'       # Rosa para mujeres
    }
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='sexo', 
                     color_discrete_map=color_map,
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>All work, no play</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Children in child labour*</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores
            dict(
                x=0.01, y=-0.15,
                text='Selected regions, % of child population',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.15,
                text="Number of children, m",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%')
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombre', 2: 'Mujer'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    color_map = {
        'Hombre': '#1f77b4',  # Azul para hombres
        'Mujer': 'pink'       # Rosa para mujeres
    }
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='sexo', 
                     color_discrete_map=color_map,
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>All work, no play</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Children in child labour*</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores
            dict(
                x=0.01, y=-0.15,
                text='Selected regions, % of child population',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.15,
                text="Number of children, m",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    color_map = {
        'Hombres': '#1f77b4',  # Azul para hombres
        'Mujeres': 'pink'       # Rosa para mujeres
    }
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='sexo', 
                     color_discrete_map=color_map,
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>All work, no play</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Children in child labour*</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores
            dict(
                x=0, y=-0.1,
                text='Selected regions, % of child population',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Number of children, m",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    color_map = {
        'Hombres': '#1f77b4',  # Azul para hombres
        'Mujeres': 'pink'       # Rosa para mujeres
    }
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='sexo', 
                     color_discrete_map=color_map,
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales y regiones
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap para hombres
def crear_treemap_hombres(distribution: pd.DataFrame) -> None:
    hombres_data = distribution[distribution['sexo'] == 'Hombres']
    fig = px.treemap(hombres_data, 
                     path=['sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  
                     color_continuous_scale='Blues',  # Escala de colores en azul
                     labels={'percentage': 'Porcentaje'})
    
    fig.update_layout(margin=dict(t=50, b=50), title="Distribución de Hombres")
    fig.show()

# Crear el treemap para mujeres
def crear_treemap_mujeres(distribution: pd.DataFrame) -> None:
    mujeres_data = distribution[distribution['sexo'] == 'Mujeres']
    fig = px.treemap(mujeres_data, 
                     path=['sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  
                     color_continuous_scale='Pinkyl',  # Escala de colores en rosado
                     labels={'percentage': 'Porcentaje'})
    
    fig.update_layout(margin=dict(t=50, b=50), title="Distribución de Mujeres")
    fig.show()

# Crear gráficos
crear_treemap_hombres(distribution)
crear_treemap_mujeres(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Blues',  # Escala de colores continua
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        margin=dict(t=100, b=100),  # Margen superior e inferior
        annotations=[
            # Título principal
            dict(
                x=0, y=1.28,  # Subir el título más arriba
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.18,  # Bajar el subtítulo más abajo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.3, x1=1, y1=1.3,  # Subir la línea más arriba
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.275, x1=0.05, y1=1.3,  # Subir el rectángulo más arriba
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.5,  # Posición horizontal centrada
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=20,  # Grosor de la barra
            len=0.5  # Largo de la barra
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.75,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=10,  # Grosor de la barra reducido
            len=0.3  # Largo de la barra reducido
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.8,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=15,  # Grosor de la barra un poco más ancho
            len=0.4,  # Largo de la barra un poco más ancho
            tickvals=[5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['5', '10', '15', '20', '25', '30', '35', '40']  # Texto de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.8,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=15,  # Grosor de la barra un poco más ancho
            len=0.4,  # Largo de la barra un poco más ancho
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%']  # Texto de los números con "%"
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.8,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=15,  # Grosor de la barra un poco más ancho
            len=0.4,  # Largo de la barra un poco más ancho
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%']  # Texto de los números con "%"
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.8,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=15,  # Grosor de la barra un poco más ancho
            len=0.4,  # Largo de la barra un poco más ancho
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%']  # Texto de los números con "%"
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=22, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=18, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=14, color="#a2a2a2", family="Econ Sans Cnd"),  # Usa la fuente solicitada
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=12,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.8,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=15,  # Grosor de la barra un poco más ancho
            len=0.4,  # Largo de la barra un poco más ancho
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=14)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=16,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.78,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=16)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + ' (' 
                                             + distribution['percentage'].astype(str) + '%)<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.78,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=16)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    # Modificación aquí: se eliminó el porcentaje en paréntesis
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.78,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=16)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    # Modificación aquí: se eliminó el porcentaje en paréntesis
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.78,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=16)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    # Modificación aquí: se eliminó el porcentaje en paréntesis
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Porcentaje'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=100),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title="Porcentaje",
            orientation='h',  # Cambia la orientación a horizontal
            x=0.78,  # Posición horizontal más a la izquierda
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0%', '5%', '10%', '15%', '20%', '25%', '30%', '35%', '40%'],  # Texto de los números con "%"
            tickfont=dict(size=16)  # Tamaño de fuente de los números
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.1,
                text='Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile',  # Anotación inferior izquierda
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.1,
                text="Nota: Datos correspondientes al proceso de admisión 2025",  # Anotación inferior derecha
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=3),
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    # Modificación aquí: se eliminó el porcentaje en paréntesis
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Distribución de estudiantes en %'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=150),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de estudiantes<br>en %</b>",  # Texto en negrita y con salto de línea
                font=dict(size=18, family="Econ Sans Cnd")  # Usar la misma fuente que los otros títulos de leyenda y aumentar el tamaño
            ),
            orientation='h',  # Cambia la orientación a horizontal
            x=0.80,  # Posición horizontal más a la derecha
            y=1.03,  # Posición vertical sobre el gráfico
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],  # Texto de los números sin "%"
            tickfont=dict(size=16),  # Tamaño de fuente de los números
            ticks="outside",  # Colocar las líneas de los ticks afuera
            tickwidth=1,  # Ancho de las líneas de los ticks
            ticklen=10  # Largo de las líneas de los ticks
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.05,  # Subir la posición del texto
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',  # Anotación inferior izquierda en negrita
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.05,  # Subir la posición del texto
                text="<b>Nota: Datos correspondientes al proceso de admisión 2025</b>",  # Anotación inferior derecha en negrita
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=1),  # Hacer la línea más delgada
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

# Ruta del archivo CSV
file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)

data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})

# Definir el mapa de las ramas educacionales
rama_educacional_mapping = {
    'H1': 'Humanista científico Diurno',
    'H2': 'Humanista científico Nocturno',
    'H3': 'Humanista científico - validación de estudios',
    'H4': 'Humanista científico - Reconocimiento de estudios',
    'T1': 'Técnico profesional comercial',
    'T2': 'Técnico profesional Industrial',
    'T3': 'Técnico profesional Servicios y técnica',
    'T4': 'Técnico profesional agrícola',
    'T5': 'Técnico profesional Marítima'
}
data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)

# Definir el mapa de las regiones
region_mapping = {
    15: 'Región de Arica y Parinacota',
    1: 'Región de Tarapacá',
    2: 'Región de Antofagasta',
    3: 'Región de Atacama',
    4: 'Región de Coquimbo',
    5: 'Región de Valparaíso',
    13: 'Región Metropolitana de Santiago',
    6: 'Región del Libertador General Bernardo O\'Higgins',
    7: 'Región del Maule',
    16: 'Región de Ñuble',
    8: 'Región del Biobío',
    9: 'Región de La Araucanía',
    14: 'Región de Los Ríos',
    10: 'Región de Los Lagos',
    11: 'Región de Aysén del General Carlos Ibáñez del Campo',
    12: 'Región de Magallanes y de la Antártica Chilena'
}
data['codigo_region'] = data['codigo_region'].map(region_mapping)

# Calcular la distribución
def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    # Modificación aquí: se eliminó el porcentaje en paréntesis
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' 
                                             + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

distribution = calcular_distribucion(data)

# Crear el treemap
def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage',  # Usar el porcentaje para el degradé de colores
                     color_continuous_scale='Viridis',  # Escala de colores continua y neutra
                     title='',  # El título se manejará en las anotaciones
                     labels={'percentage': 'Distribución de estudiantes en %'},
                     hover_data={'count': True})
    
    # Ajustar el margen para hacer espacio para el texto superior
    fig.update_layout(
        height=1000,  # Ajustar la altura del gráfico
        margin=dict(t=200, b=150),  # Aumentar el margen superior e inferior
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de<br>estudiantes en %</b>",  # Texto en negrita y con salto de línea
                font=dict(size=18, family="Econ Sans Cnd")  # Usar la misma fuente que los otros títulos de leyenda y aumentar el tamaño
            ),
            orientation='h',  # Cambia la orientación a horizontal
            x=0.79,  # Posición horizontal más a la izquierda
            y=1.02,  # Posición vertical más arriba
            thickness=17,  # Grosor de la barra un poco más ancho
            len=0.42,  # Largo de la barra un poco más ancho
            tickvals=[0, 5, 10, 15, 20, 25, 30, 35, 40],  # Números en la barra de porcentaje
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],  # Texto de los números sin "%"
            tickfont=dict(size=16),  # Tamaño de fuente de los números
            ticks="outside",  # Colocar las líneas de los ticks afuera
            tickwidth=1,  # Ancho de las líneas de los ticks
            ticklen=10  # Largo de las líneas de los ticks
        ),
        annotations=[
            # Título principal
            dict(
                x=0, y=1.15,  # Ajustar la posición del título
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",  # Texto del título en negrita
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Subtítulo
            dict(
                x=0, y=1.10,  # Ajustar la posición del subtítulo
                text="<b>Análisis de Admisiones PAES 2025</b>",  # Texto del subtítulo en negrita
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            # Fuente y anotaciones inferiores usando la fuente solicitada
            dict(
                x=0, y=-0.05,  # Subir la posición del texto
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',  # Anotación inferior izquierda en negrita
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.05,  # Subir la posición del texto
                text="<b>Nota: Datos correspondientes al proceso de admisión 2025</b>",  # Anotación inferior derecha en negrita
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),  # Aumentar el tamaño de la fuente
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            # Línea horizontal en la parte superior
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,  # Ajustar la posición de la línea
                line=dict(color="red", width=1),  # Hacer la línea más delgada
                xref="paper", yref="paper"
            ),
            # Rectángulo en la parte superior izquierda
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,  # Ajustar la posición del rectángulo
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    # Ajustar el tamaño mínimo de los rectángulos y usar el algoritmo "squarify"
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,  # Aumentar el tamaño de la fuente de los textos en los rectángulos
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

# Crear el gráfico
crear_treemap(distribution)

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def cargar_datos(file: str) -> pd.DataFrame:
    return pd.read_csv(file)

def mapear_columnas(data: pd.DataFrame) -> pd.DataFrame:
    data['sexo'] = data['sexo'].map({1: 'Hombres', 2: 'Mujeres'})
    
    rama_educacional_mapping = {
        'H1': 'Humanista científico Diurno',
        'H2': 'Humanista científico Nocturno',
        'H3': 'Humanista científico - validación de estudios',
        'H4': 'Humanista científico - Reconocimiento de estudios',
        'T1': 'Técnico profesional comercial',
        'T2': 'Técnico profesional Industrial',
        'T3': 'Técnico profesional Servicios y técnica',
        'T4': 'Técnico profesional agrícola',
        'T5': 'Técnico profesional Marítima'
    }
    data['rama_educacional'] = data['rama_educacional'].map(rama_educacional_mapping)
    
    region_mapping = {
        15: 'Región de Arica y Parinacota',
        1: 'Región de Tarapacá',
        2: 'Región de Antofagasta',
        3: 'Región de Atacama',
        4: 'Región de Coquimbo',
        5: 'Región de Valparaíso',
        13: 'Región Metropolitana de Santiago',
        6: 'Región del Libertador General Bernardo O\'Higgins',
        7: 'Región del Maule',
        16: 'Región de Ñuble',
        8: 'Región del Biobío',
        9: 'Región de La Araucanía',
        14: 'Región de Los Ríos',
        10: 'Región de Los Lagos',
        11: 'Región de Aysén del General Carlos Ibáñez del Campo',
        12: 'Región de Magallanes y de la Antártica Chilena'
    }
    data['codigo_region'] = data['codigo_region'].map(region_mapping)
    
    return data

def calcular_distribucion(data: pd.DataFrame) -> pd.DataFrame:
    distribution = data.groupby(['sexo', 'rama_educacional', 'codigo_region']).size().reset_index(name='count')
    total_by_sex = distribution.groupby('sexo')['count'].transform('sum')
    distribution['percentage'] = round((distribution['count'] / total_by_sex) * 100, 2)
    distribution['label_with_percentage'] = (distribution['rama_educacional'] + '<br>' + distribution['codigo_region'])
    distribution['general_label'] = 'Distribución de estudiantes por sexo'
    return distribution

def crear_treemap(distribution: pd.DataFrame) -> None:
    fig = px.treemap(distribution, 
                     path=['general_label', 'sexo', 'label_with_percentage'], 
                     values='percentage', 
                     color='percentage', 
                     color_continuous_scale='Viridis', 
                     title='', 
                     labels={'percentage': 'Distribución de estudiantes en %'},
                     hover_data={'count': True})
    
    fig.update_layout(
        height=1000,
        margin=dict(t=200, b=150),
        coloraxis_colorbar=dict(
            title=dict(
                text="<b>Distribución de<br>estudiantes en %</b>",
                font=dict(size=18, family="Econ Sans Cnd")
            ),
            orientation='h',
            x=0.79,
            y=1.02,
            thickness=17,
            len=0.42,
            tickvals=[1, 5, 10, 15, 20, 25, 30, 35, 40],
            ticktext=['0', '5', '10', '15', '20', '25', '30', '35', '40'],
            tickfont=dict(size=16),
            ticks="outside",
            tickwidth=1,
            ticklen=10
        ),
        annotations=[
            dict(
                x=0, y=1.15,
                text="<b>Distribución de Estudiantes por Sexo, Rama Educacional y Región</b>",
                showarrow=False,
                font=dict(size=26, color="black", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            dict(
                x=0, y=1.10,
                text="<b>Análisis de Admisiones PAES 2025</b>",
                showarrow=False,
                font=dict(size=22, color="gray", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="top"
            ),
            dict(
                x=0, y=-0.05,
                text='<b>Fuente: Datos de Admisiones PAES 2025, Ministerio de Educación de Chile</b>',
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="left", yanchor="bottom"
            ),
            dict(
                x=1, y=-0.05,
                text="<b>Nota: Datos correspondientes al proceso de admisión 2025</b>",
                showarrow=False,
                font=dict(size=18, color="#a2a2a2", family="Econ Sans Cnd"),
                xref="paper", yref="paper",
                xanchor="right", yanchor="bottom"
            )
        ],
        shapes=[
            dict(
                type="line",
                x0=0, y0=1.17, x1=1, y1=1.17,
                line=dict(color="red", width=1),
                xref="paper", yref="paper"
            ),
            dict(
                type="rect",
                x0=0, y0=1.145, x1=0.05, y1=1.17,
                line=dict(color="red", width=0),
                fillcolor="red",
                xref="paper", yref="paper"
            )
        ]
    )
    
    fig.update_traces(marker=dict(line=dict(width=2)),
                      textinfo='label+text+value',
                      textfont_size=18,
                      texttemplate='%{label}<br>%{value}%',
                      tiling=dict(pad=1, squarifyratio=1.0))
    
    fig.show()

file_path = 'C:/Users/BS/Documents/PAES2025_Admissions_Analysis/data/processed/cleaned_admissions_data_2025.csv'
data = cargar_datos(file_path)
data = mapear_columnas(data)
distribution = calcular_distribucion(data)
crear_treemap(distribution)